In [30]:
import os
from dotenv import load_dotenv

In [31]:
load_dotenv('./.env')
os.environ.get('PINECONE_ENV')

'gcp-starter'

* Instantiating the API_KEY

In [32]:
OPENAI_API_KEY = os.getenv('OPENAI_SECREY_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_ENV')

## LangChain getting started


In [33]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512, openai_api_key=OPENAI_API_KEY)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


* Asking AI a question.

In [34]:
output = llm('explique como funções python funcionam')

In [35]:
print(output)



Funções Python são blocos de código que executam uma tarefa específica quando são chamados. Elas são definidas usando a palavra-chave def. Uma função recebe argumentos e, opcionalmente, retorna um valor. Funções Python são usadas para modularizar a lógica do programa, tornando mais fácil de ler e debugar o código. Uma função também pode ser chamada repetidamente para executar a mesma tarefa várias vezes, tornando o código mais eficiente.


* Count Tokens

In [36]:
print(llm.get_num_tokens('explique como funções python funcionam'))

13


* list of questions and answers

In [37]:
output = llm.generate(['... é a capital da França',
                       'qual é a formula da área de um círculo?'])

In [38]:
print(output.generations)

[[Generation(text='?\n\nParis.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nA área de um círculo é dada por A = π.r², onde r é o raio do círculo.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [39]:
output.generations[0][0].text

'?\n\nParis.'

In [40]:
output = llm.generate(['escreva um slogan original para um restaurante de sushi'] * 3)

In [41]:
for o in output.generations:
    print(o[0].text, end='')



"Sushi com sabor de autenticidade: Venha sentir o verdadeiro sabor do Oriente!"

"A frescura do sabor japonês, no coração da cidade!"

"Sushi com sabores que vão te surpreender!"